<a href="https://colab.research.google.com/github/sadrakhamoshi/CI-Course/blob/main/CI992_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your info

Full name: Sadra Khamoshi 

Student ID: 97521261


# Q1. Hopfield

## Q1.2

In [ ]:
# Q1.2_graded
# Do not change the above line.

# This cell is for your imports.

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Q1.2_graded
# Do not change the above line.

# This cell is for your codes.

patterns = np.array([[1, -1, 1, -1, 1, -1], [1, 1, 1, -1, -1, -1]])
patterns_T = patterns.T
weights = np.dot(patterns_T, patterns)
np.fill_diagonal(weights, 0)
print(f'weights : \n {repr(weights)}')


weights : 
 array([[ 0,  0,  2, -2,  0, -2],
       [ 0,  0,  0,  0, -2,  0],
       [ 2,  0,  0, -2,  0, -2],
       [-2,  0, -2,  0,  0,  2],
       [ 0, -2,  0,  0,  0,  0],
       [-2,  0, -2,  2,  0,  0]])


In [ ]:
# Q1.2_graded
# Do not change the above line.

# This cell is for your codes.

x = [-1, 1, 1, -1, -1, -1]

def compute_new_x(x, index):
        fire = np.sum(x*weights[index])
        if fire >= 0:
            x[index] = 1
        else:
            x[index] = -1
        return x
  
last_x = np.copy(x)
for i in range(1000):
  x = compute_new_x(x, i % 6)
  if (i + 1) % 6 == 0:
      if np.array_equal(last_x, x):
          break
      last_x = np.copy(x)
print(f'result : {repr(x)}')

result : [1, 1, 1, -1, -1, -1]


## Q1.3

In [ ]:
# Q1.3_graded
# Do not change the above line.

# This cell is for your imports.
from PIL import Image, ImageFont
import os
import numpy as np
import random
from pathlib import Path


In [ ]:
# Q1.3_graded
# Do not change the above line.

# This cell is for your codes.
# generate_data

path = '/content/Pictures/'
chars = 'ABCDEFGHIJ'
fonts = [16, 32, 64]
noises = [0.1, 0.3, 0.6]


def generate_correct_data(threshold=0):
    for f in fonts:
        font = ImageFont.truetype('/content/tahoma.ttf', f)
        images = {}
        y_max =0
        x_max =0 
        for ch in chars:
            im = Image.Image()._new(font.getmask(ch))
            y, x = im.size
            if x > x_max:
                x_max = x
            if y > y_max:
                y_max = y
            images[f'{ch}_{str(f)}'] = im        
        
        for k,v in images.items():
            im_arr = np.array(v)
            shape = im_arr.shape
            offset_x = int((x_max - shape[0]) / 2)
            offset_y = int((y_max - shape[1]) / 2)
            
            matrix_layout = np.zeros((x_max, y_max), dtype="uint8")
            matrix_layout[offset_x:offset_x + shape[0], offset_y:offset_y + shape[1]] = im_arr
            
            im_arr = matrix_layout
            s = np.where(im_arr > 0 , 255, 0)
            im_fixed = Image.fromarray(np.array(s,dtype=np.uint8))
            im_fixed.save(f'{path}correct_data/{str(f)}/{k}.bmp')


def add_noise_v2(img_arr, noise, fp, fontsize):
    all_pixels = int(img_arr.shape[0]) * int(img_arr.shape[1])
    noisy_pixels_num = int(all_pixels * noise)
    noisy_image = np.copy(img_arr)
    random_values = random.sample(range(0, all_pixels), noisy_pixels_num)

    for val in random_values:
        row = int(val / img_arr.shape[1])
        col = int(val % img_arr.shape[1])
        if img_arr[row][col] == 255:
            noisy_image[row][col] = 0
        else:
            noisy_image[row][col] = 255

    img = Image.fromarray(noisy_image, mode='L')
    noise = int(noise * 100)
    img.save(f'{path}noisy_data/{str(noise)}/{str(fontsize)}/{fp}')


def generate_noisy_data(fontsize):
    dir = os.listdir(f'/content/Pictures/correct_data/{str(fontsize)}')
    for no in noises:
        for fp in dir:
            img = Image.open(f'/content/Pictures/correct_data/{str(fontsize)}/{fp}', mode='r')
            img_arr = np.array(img)
            add_noise_v2(img_arr, no, fp, fontsize)

def generate_noise():
    for f in fonts:
        generate_noisy_data(f)

def main():
    # use below line to generate data without noise
    generate_correct_data()

    # use below line to generate data with noise
    generate_noise()

def create_directory():
  if not os.path.isdir(f'{path}'):
    for f in fonts:
      os.makedirs(f'{path}correct_data/{str(f)}')

  if not os.path.isdir(f'{path}noisy_data'):
    for n in noises:
      for f in fonts:
        os.makedirs(f'{path}noisy_data/{str(int(n*100))}/{str(f)}')
      os.makedirs(f'{path}noisy_data/results')


if __name__ == '__main__':
    create_directory()
    main()


In [ ]:
# Q1.3_graded
# Do not change the above line.

# This cell is for your codes.
# hopfield Netwrok

class hopfield:
    def __init__(self, n_num):
        self.num = n_num
        self.weights = np.zeros((n_num, n_num))

    def train(self, patterns, types='bipolar'):
        patterns_T = patterns.T
        self.weights = np.dot(patterns_T, patterns)
        np.fill_diagonal(self.weights, 0)
        return

    def compute_new_x(self, x, index):
        fire = np.sum(x * self.weights[index])
        if fire >= 0:
            x[index] = 1
        else:
            x[index] = -1
        return x

    def async_update(self, x, max_itr=1000):
        last_x = np.copy(x)
        for i in range(max_itr):
            x = self.compute_new_x(x, i % self.num)
            if (i + 1) % self.num == 0:
                if np.array_equal(last_x, x):
                    return x
                last_x = np.copy(x)
        return x

    def sync_update(self, x):
        last_x = np.copy(x)
        for i in range(1000):
            x = self.restore(x)
            if np.array_equal(x, last_x):
                break
            last_x = np.copy(x)
        return x

    def sign(self, t):
        t = np.where(t >= 0, 1, -1)
        return t

    def restore(self, x):
        t = np.sum(x * self.weights, axis=1)
        return np.array(self.sign(t.T))

    def evaluate_network(self, x):
        return np.sign(np.dot(self.weights, x.T))


def get_patterns():
    patterns = []
    dim1 = 48
    dim2 = 50
    images = os.listdir(f'New folder/images/')
    for img in images:
        tmp = Image.open(f'images/{img}', mode='r')
        tmp = tmp.resize((dim1, dim2))
        tmp = np.asarray(tmp, dtype=np.uint8)
        pattern = -1 * np.ones(tmp.shape)
        pattern[tmp > 60] = 1
        patterns.append(pattern.flatten())
    return np.array(patterns)


def get_train_data(font):
    patterns = []
    path = f'/content/Pictures/correct_data/{str(font)}/'
    files = os.listdir(path)
    max_x = 0
    max_y = 0
    for file in files:
        tmp = Image.open(f'{path}{file}', mode='r')
        if tmp.size[1] > max_x:
            max_x = tmp.size[1]
        if tmp.size[0] > max_y:
            max_y = tmp.size[0]
        tmp.close()

    for file in files:
        tmp = Image.open(f'{path}{file}', mode='r')
        img_array = tmp.resize((max_x, max_y))
        img_array = np.asarray(img_array, dtype=np.uint8)
        pattern = -1 * np.ones(img_array.shape)
        pattern = np.where(img_array == 255, 1, -1)
        patterns.append(pattern.flatten())
    return np.array(patterns), (max_x, max_y)


def get_train_data_v2(font):
    patterns = []
    path = f'/content/Pictures/correct_data/{str(font)}/'
    files = os.listdir(path)
    for file in files:
        tmp = Image.open(f'{path}{file}', mode='r')
        img_array = np.asarray(tmp, dtype=np.uint8)
        pattern = -1 * np.ones(img_array.shape)
        pattern = np.where(img_array == 255, 1, -1)
        patterns.append(pattern.flatten())
        ss = len(pattern.flatten())
    return np.array(patterns)


def evaluate_hopfield(hp: hopfield, noise, font , con):
    acc_evaluate = []
    acc_converge = []
    path = f'/content/Pictures/noisy_data/{str(noise)}/{str(font)}/'
    res_path = f'/content/Pictures/noisy_data/{str(noise)}/results/'
    correct_data = f'/content/Pictures/correct_data/{str(font)}/'
    files = os.listdir(path)
    for file in files:
        tmp = Image.open(f'{path}{file}', mode='r')
        # img_array = tmp.resize((size[0], size[1]))
        img_array = np.asarray(tmp, dtype=np.uint8)
        shape = img_array.shape
        pattern = -1 * np.ones(img_array.shape)
        pattern = np.where(img_array == 255, 1, -1)
        flatten_pattern = pattern.flatten()

        fixed_img_arr = hp.evaluate_network(flatten_pattern)

        converge = hp.sync_update(flatten_pattern)

        fixed_img_arr = np.where(fixed_img_arr == 1, 255, 0)
        fixed_img_arr = np.array(fixed_img_arr, dtype=np.uint8)

        ## converge
        converge = np.where(converge == 1, 255, 0)
        converge = np.array(converge, dtype=np.uint8)

        ## compute accuracy
        correct_img = Image.open(f'{correct_data}{file}')
        correct_img_arr = np.asarray(correct_img, dtype=np.uint8).flatten()
        correctness = np.sum(fixed_img_arr == correct_img_arr)
        acc_evaluate.append(correctness / len(correct_img_arr))

        ## converge accuracy
        correctness_con = np.sum(converge == correct_img_arr)
        acc_converge.append(correctness_con / len(correct_img_arr))

        res_img = Image.fromarray(fixed_img_arr.reshape(shape), mode='L')
        res_img.save(f'{res_path}{str(noise)}_{file}')

    print(
        f'font : {font} , noise : {noise} ====> Evaluate accuracy : {np.mean(acc_evaluate)}'
    )
    con.append(f'font : {font} , noise : {noise} ====> Converge accuracy : {np.mean(acc_converge)}')


def main():
    fontsizes = [16, 32, 64]
    noises = [10, 30, 60]
    sizes = [11, 23, 45]
    chars = 'ABCDEFGHIJ'
    con = []

    for font in fontsizes:
        patterns = get_train_data_v2(font)
        neurons = len(patterns[0])
        hp = hopfield(int(neurons))
        hp.train(patterns)
        for noise in noises:
            evaluate_hopfield(hp, noise, font, con)

    print('convergence accuracy : ')
    for c in con:
        print(c)

if __name__ == '__main__':
    main()

font : 16 , noise : 10 ====> Evaluate accuracy : 0.929861111111111
font : 16 , noise : 30 ====> Evaluate accuracy : 0.9430555555555555
font : 16 , noise : 60 ====> Evaluate accuracy : 0.1375
font : 32 , noise : 10 ====> Evaluate accuracy : 0.9359683794466405
font : 32 , noise : 30 ====> Evaluate accuracy : 0.9282608695652174
font : 32 , noise : 60 ====> Evaluate accuracy : 0.09426877470355732
font : 64 , noise : 10 ====> Evaluate accuracy : 0.9102990033222591
font : 64 , noise : 30 ====> Evaluate accuracy : 0.900047460844803
font : 64 , noise : 60 ====> Evaluate accuracy : 0.10783103939250119
convergence accuracy : 
font : 16 , noise : 10 ====> Converge accuracy : 0.7402777777777778
font : 16 , noise : 30 ====> Converge accuracy : 0.7402777777777778
font : 16 , noise : 60 ====> Converge accuracy : 0.25972222222222224
font : 32 , noise : 10 ====> Converge accuracy : 0.8446640316205534
font : 32 , noise : 30 ====> Converge accuracy : 0.8031620553359684
font : 32 , noise : 60 ====> Conver

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='green'>**D5372R**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
    - **Note**: if you want to add more cells, add this to the **first** line of the cell:
  ```python
  # Q5_graded
  ```
  according to the question number.
  - There are 2 assignments in Gradescope:

    ![assignments](https://i.ibb.co/10GMhGM/Screen-Shot-2021-03-21-at-02-16-25.png)
  
    You should upload your **codes** and your **notebook** in `HW3` section and your final report for all of the questions as a **single pdf** file in `HW3 - Report`. Autograder will automatically check for:
    - `CI992_HW3.ipynb`
    - `Q1.2.py`
    - `Q1.3.py`
    - `inverted_pendulum.fcl`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.